In [1]:
#Better
!pip install requests BeautifulSoup4 fire

     |████████████████████████████████| 81kB 2.1MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=1a23d2e3e9b665ff5124f20181d40485be7daa6ddddcf05d79924224f0796fbc
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [3]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
def get_element(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url
    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
        if search:
            soup = html            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
        return res
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

Web scrape 1000 most influential Twitter users in Africa using Python or Bash to obtain the 100 African twitter influencers.

You are required to compile active twitter users' handles for your country. You can start
doing so by collecting first the twitter accounts of

● Top government officials including ministries and deputy ministers

● Active journalists and their media network

● Active ambassadors

● Active celebrities

● List of 
Top government officials including ministries and deputy ministers

In [4]:
import tweepy
import csv

consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

In [5]:
res = get_elements('https://www.intelligencefusion.co.uk/blog/top-30-most-followed-news-accounts-on-twitter',tag='p')
s = str(res)

import re
extracted_newslink = re.findall(r"@(\w+)", s)
str(extracted_newslink)

"['telegraph', 'telegraph', 'cbcnews', 'cbcnews', 'independent', 'independent', 'rt_com', 'rt_com', 'guardiannews', 'guardiannews', 'france24_en', 'france24_en', 'france24', 'france24', 'cnbc', 'cnbc', 'newsweek', 'newsweek', 'skynewsbreak', 'skynewsbreak', 'ajenglish', 'ajenglish', 'skynews', 'skynews', 'financialtimes', 'financialtimes', 'guardian', 'guardian', 'breakingnews', 'breakingnews', 'huffpost', 'huffpost', 'ndtv', 'ndtv', 'xhnews', 'xhnews', 'ap', 'ap', 'washingtonpost', 'washingtonpost', 'abc', 'abc', 'time', 'time', 'wsj', 'wsj', 'reuters', 'reuters', 'theeconomist', 'theeconomist', 'bbcworld', 'bbcworld', 'bbcbreaking', 'bbcbreaking', 'cnn', 'cnn', 'nytimes', 'nytimes', 'cnnbrk', 'cnnbrk', 'IntellFusion', 'IFNorthAmerica', 'IFSouthAmerica', 'IF_Africa', 'IF_Asia_', 'IF_Europe']"

In [6]:
url= 'https://www.okayafrica.com/top-15-nigerian-political-twitter-accounts/'
top_govOfficials = []
response = get(url).content
res_gov = get_elements(response, tag='blockquote')
gov_officials_twitter_handles = []
for r in res_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    gov_officials_twitter_handles.append(handle)
    top_govOfficials.append(handle)
    print(f'{name} : {handle}')

Zainab Usman : @Zainab Usman
Zainab Usman : @Zainab Usman
Zainab Usman : @Zainab Usman
JJ. Omojuwa : @JJ. Omojuwa
Gimba Kakanda : @Gimba Kakanda
Aisha Yesufu : @Aisha Yesufu
AYO SOGUNRO : @AYO SOGUNRO
Mr. Aye Dee : @Mr. Aye Dee
Mr. Aye Dee : @Mr. Aye Dee
'Gbénga Sèsan : @'Gbénga Sèsan
The Godfather : @The Godfather
Khaleesi : @Khaleesi
Chidi Okereke : @Chidi Okereke
Chidi Okereke : @Chidi Okereke
Ada Ọpia Egbe : @Ada Ọpia Egbe
Rosanwo : @Rosanwo
Rosanwo : @Rosanwo
HENRY Okelue : @HENRY Okelue
Onye Nkuzi : @Onye Nkuzi
Onye Nkuzi : @Onye Nkuzi
Demola Olarewaju : @Demola Olarewaju
Demola Olarewaju : @Demola Olarewaju


In [7]:
# The Twitter user who we want to get followers count for
  # Calling the user_timeline function with our parameters
govofficial_list = ["@seyiamakinde","@officialKolaO","@RotimiAkeredolu"]
journalist_list = ["@channelstv","@TVCconnect"]
celebrities_list = ["@topealabi01", "@funkeakindele"]

RotimiAkeredolu = []
seyiamakinde = []
TVCconnect = []
channelstv = []
funkeakindele = []
topealabi01 = []
ayosogunro = []

In [8]:
for user in tweepy.Cursor(api.friends, screen_name="@RotimiAkeredolu").items():
  RotimiAkeredolu.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@seyiamakinde").items():
  seyiamakinde.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@funkeakindele").items():
  funkeakindele.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@TVCconnect").items():
  TVCconnect.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@topealabi01").items():
  topealabi01.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@channelstv").items():
  channelstv.append(user.screen_name)

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="@ayosogunro").items():
  ayosogunro.append(user.screen_name)

In [10]:
seyiamakinde = ['Bolasarumi_ali', 'RotimiAkeredolu', 'Seun_TNA', 'FunmiSalami3', 'BVA_Bello', 'AdeniyiAdebisi_', 'JideAdeosun', 'jumokeakinjide', 'PeterObi', 'MoghaluKingsley', 'TheSeunFakorede', 'bukolasaraki', 'iamakinolaojo', 'IsiakaAdeleke1', 'iamHSDickson', 'akande_debo', 'DAWNCommission', 'iamominini', 'oyostategovt', 'raufolaniyan', 'PDPOyoState', 'ProfDKSH', 'OID_Global', 'Adebo_ogund', 'ilaka_bisi', 'abassade', 'BudgITng', 'thelionheart_t', 'captainbebo', 'GeniusJokerNG', 'Omoboinke1', 'D_GeneralMonroe', 'SayyuAminu', 'AkinAdegboyega', 'MatthewOgunlere', 'gyonlineng', 'Mayorduke1', 'darlingbayo', 'PDP_2023', 'SupportSeyi', 'PDPVanguard', 'iam_ykay', 'alagbafo1hour', 'IbadanGotTalent', 'makinwaoluwole', 'fmlykdat', 'GoodluckNgr', '1race_tv', 'The_Osazuwa', 'Sopirinye', 'AkanbiOri', 'ODUXOLOGY', 'teejay_tobman', 'seunomoakinola', 'rollybello', 'DrAgboEmmanuel', 'GovAyoFayose', 'HECTVofficial', 'atiku', 'OfficialPDPNig', 'ayoola_iyanda', 'musliudeen', 'seundavid0310', 'Abdulfatihoba', 'neyopompe01', 'richardmammah', 'ola_shodimu', 'akeem_lasisi', 'Starfm915', 'mulikatadeola', 'ibcityannouncer', 'freshfmibadan', 'eDoctor7', 'oladipokunle', 'delaw1999', 'PowemIbadan', 'mike_imouk', 'echannelhex', 'theayogeorge', 'seunonigbinde', 'AyoAyodabo1', 'molarawood', 'yppjf', 'akande_adebayo', 'kehindeayoolajp', 'Isrimmsky1', 'TheCakeChancery', 'WaleOzolua', 'MrKakaki', 'ekufre', 'GovernorKaslas', 'TayePaulOlubayo', 'bolu_oj', 'Adetomilayo', 'Tolu_Erogbogbo', 'NejeebBello', 'iamtohlublaize', 'UnibadanGossips', 'wildeyeq', 'borie_nla', 'legitngnews', 'dsalahu', 'UcheBriggs', 'amaroq_ilaya', 'JauntieBellz', 'oluwajuwalo', 'Olurotimisure', 'Warlorrd70', 'linbizglobal', 'tobaranest', 'ogunmilorokenny', 'observer_2027', 'leaderwolf14', 'ekajnr', 'nujenews', 'rossonero101', 'Beatpluz', 'alaoadams', 'olaniyan040', 'OLANIKEEMII', 'ehyitayo', 'leraygunn', 'abayoleye2007', 'tundeo514', 'texglobals', 'dutem18', 'PromptNewso', 'iamtennygee', 'llb60', 'RGAMeyer', 'lagosbooksclub', 'PrinceofSydney', 'EletikoO', 'Remadebayo', 'femona2tawa', 'PHEMY37', 'Mattoo198', 'PeterEledan', 'Goldensax', 'AkinjideAjisafe', 'Rem_MankinD', 'dejiSHALOM', 'Ii_zik', 'tem2202', 'FEGO_OS', 'Funkyshinaayo', 'Omobaale', 'TheIbadanCity', 'OlaWayn', 'whatsupibadan', 'feelib', 'Ibadan247', 'Bolasarumi_ali', 'RotimiAkeredolu', 'Seun_TNA', 'FunmiSalami3', 'BVA_Bello', 'AdeniyiAdebisi_', 'JideAdeosun', 'jumokeakinjide', 'PeterObi', 'MoghaluKingsley', 'TheSeunFakorede', 'bukolasaraki', 'iamakinolaojo', 'IsiakaAdeleke1', 'iamHSDickson', 'akande_debo', 'DAWNCommission', 'iamominini', 'oyostategovt', 'raufolaniyan', 'PDPOyoState', 'ProfDKSH', 'OID_Global', 'Adebo_ogund', 'ilaka_bisi', 'abassade', 'BudgITng', 'thelionheart_t', 'captainbebo', 'GeniusJokerNG', 'Omoboinke1', 'D_GeneralMonroe', 'SayyuAminu', 'AkinAdegboyega', 'MatthewOgunlere', 'gyonlineng', 'Mayorduke1', 'darlingbayo', 'PDP_2023', 'SupportSeyi', 'PDPVanguard', 'iam_ykay', 'alagbafo1hour', 'IbadanGotTalent', 'makinwaoluwole', 'fmlykdat', 'GoodluckNgr', '1race_tv', 'The_Osazuwa', 'Sopirinye', 'AkanbiOri', 'ODUXOLOGY', 'teejay_tobman', 'seunomoakinola', 'rollybello', 'DrAgboEmmanuel', 'GovAyoFayose', 'HECTVofficial', 'atiku', 'OfficialPDPNig', 'ayoola_iyanda', 'musliudeen', 'seundavid0310', 'Abdulfatihoba', 'neyopompe01', 'richardmammah', 'ola_shodimu', 'akeem_lasisi', 'Starfm915', 'mulikatadeola', 'ibcityannouncer', 'freshfmibadan', 'eDoctor7', 'oladipokunle', 'delaw1999', 'PowemIbadan', 'mike_imouk', 'echannelhex', 'theayogeorge', 'seunonigbinde', 'AyoAyodabo1', 'molarawood', 'yppjf', 'akande_adebayo', 'kehindeayoolajp', 'Isrimmsky1', 'TheCakeChancery', 'WaleOzolua', 'MrKakaki', 'ekufre', 'GovernorKaslas', 'TayePaulOlubayo', 'bolu_oj', 'Adetomilayo', 'Tolu_Erogbogbo', 'NejeebBello', 'iamtohlublaize', 'UnibadanGossips', 'wildeyeq', 'borie_nla', 'legitngnews', 'dsalahu', 'UcheBriggs', 'amaroq_ilaya', 'JauntieBellz', 'oluwajuwalo', 'Olurotimisure', 'Warlorrd70', 'linbizglobal', 'tobaranest', 'ogunmilorokenny', 'observer_2027', 'leaderwolf14', 'ekajnr', 'nujenews', 'rossonero101', 'Beatpluz', 'alaoadams', 'olaniyan040', 'OLANIKEEMII', 'ehyitayo', 'leraygunn', 'abayoleye2007', 'tundeo514', 'texglobals', 'dutem18', 'PromptNewso', 'iamtennygee', 'llb60', 'RGAMeyer', 'lagosbooksclub', 'PrinceofSydney', 'EletikoO', 'Remadebayo', 'femona2tawa', 'PHEMY37', 'Mattoo198', 'PeterEledan', 'Goldensax', 'AkinjideAjisafe']

In [11]:
RotimiAkeredolu = ['ShinaPeller', 'ODIRS_NG', 'NigeriaNewsdesk', 'OndoAssembly', 'IAOkowa', 'mrmacaronii', 'TeniEntertainer', 'Saheed_Ibrahim1', 'StephanieBusari', 'dawisu', 'channelstv', 'PINDFoundation', 'YulEdochie', 'Moyinolubunmi', 'AusHCAbuja', 'stefanswartpet', 'thecableng', 'legitngnews', 'kola_aina', 'PremiumTimesng', 'LSMOH', 'ProfAkinAbayomi', 'Abakinruntan', 'cenbank', 'Rebecca_Huey', 'jimidisu', 'YemieFASH', 'UBAFoundation', 'UBAGroup', 'MrTomide', 'Fmohnigeria', 'Chikwe_I', 'NCDCgov', 'OvieOmoAgege', 'OndoFirstBorn', 'GovUmarGanduje', 'GbemiOO', '1babse', 'emPawaAfrica', 'Leopardybn', 'oyostategovt', 'seyiamakinde', 'SenUcheEkwunife', 'BudgITng', 'emergingtechAF', 'ClemAgba', 'TrackaNG', 'CatrionaLaing1', 'ObatAkinruntan', 'IBMMEA', 'IBM', 'officialEFCC', 'jidesanwoolu', 'PGFNigeria', 'GarShehu', 'bisiafayemi', 'BlessedMichae19', 'RussellOkung', 'OvieAli', 'mreazi', 'Innovate_Ondo', 'Akure_techhub', 'markessien', 'OndoRural', 'AfDB_Group', 'TheCitadelNg', 'akin_adesina', 'OfficialAPCNg', 'allensowore451', 'seunokin', 'soledadobrien', 'HMAdebayoShittu', 'UNICEF_Nigeria', 'Buddyrichy141', 'osrctvondo', 'NigeriaGov', 'NGRPresident', 'osphcdb', 'OlumbeAKinkugbe', 'Nairametrics', 'followlasg', 'obyezeks', 'DrJoeAbah', 'UKinNigeria', 'SunshineFCAkure', 'MADENigerDelta', 'NsimaUEkere', 'LaureBeaufils', 'DAWNCommission', 'OndoStateHealth', 'SFHNigeria', 'floondo', 'D_Aruwajoye', 'Segunmimiko', 'realRochas', 'bonifacemwangi', 'LolaAkinmade', 'MNDA_Ng', 'dareynow', 'Realomosexy', 'daily_trust', 'ogundamisi', 'EiENigeria', 'SanRomeo', 'AATSarumi', 'bukolasaraki', 'PentagonMG', 'YeleSowore', 'NigerianLaw', 'NigBarAssoc', 'GovernorObaseki', 'FAdekoyaSAN', 'DeleMomodu', 'teamaketi', 'elrufai', 'OndoAPC', 'Adaba889fm', 'BukieAdetula', 'SenBoroffice', 'oandalegal', 'femigbaja', 'AkinwunmiAmbode', 'AlegehAugustine', 'APCNigeria', 'ProfOsinbajo', 'MBuhari', 'atiku', 'iamtokunbojames', 'adetai', 'AsiwajuTinubu', 'deluchi2001', 'abikedabiri', 'GbenroAdegbola', 'TRLPLAW', 'SundayDareSD', 'kunlf', 'looceye', 'Bettyaak', 'Elombah', 'PolarisBankLtd', 'dichefit', 'steveikeomumu', 'TomiDee', 'KolawoleAketi', 'YinkaParrot', 'tfolaadeola', 'lanreneville', 'SIAmosun', 'OndoForTruth', 'MobilePunch', 'Unotech', 'OndoStateToday', 'kfayemi', 'kayodea', 'tundefashola', 'AAAjimobi', 'raufaregbesola', 'ekitistategov', 'ShinaPeller', 'ODIRS_NG', 'NigeriaNewsdesk', 'OndoAssembly', 'IAOkowa', 'mrmacaronii', 'TeniEntertainer', 'Saheed_Ibrahim1', 'StephanieBusari', 'dawisu', 'channelstv', 'PINDFoundation', 'YulEdochie', 'Moyinolubunmi', 'AusHCAbuja', 'stefanswartpet', 'thecableng', 'legitngnews', 'kola_aina', 'PremiumTimesng', 'PremiumTimesng', 'LSMOH', 'ProfAkinAbayomi', 'Abakinruntan', 'cenbank', 'Rebecca_Huey', 'jimidisu', 'YemieFASH', 'UBAFoundation', 'UBAGroup', 'MrTomide', 'Fmohnigeria', 'Chikwe_I', 'NCDCgov', 'OvieOmoAgege', 'OndoFirstBorn', 'GovUmarGanduje', 'GbemiOO', '1babse', 'emPawaAfrica', 'Leopardybn', 'oyostategovt', 'seyiamakinde', 'SenUcheEkwunife', 'BudgITng', 'emergingtechAF', 'ClemAgba', 'TrackaNG', 'CatrionaLaing1', 'ObatAkinruntan', 'IBMMEA', 'IBM', 'officialEFCC', 'jidesanwoolu', 'PGFNigeria', 'GarShehu', 'bisiafayemi', 'BlessedMichae19', 'RussellOkung', 'OvieAli', 'mreazi', 'Innovate_Ondo', 'Akure_techhub', 'markessien', 'OndoRural', 'AfDB_Group', 'TheCitadelNg', 'akin_adesina', 'OfficialAPCNg', 'allensowore451', 'seunokin', 'soledadobrien', 'HMAdebayoShittu', 'UNICEF_Nigeria', 'Buddyrichy141', 'osrctvondo', 'NigeriaGov', 'NGRPresident', 'osphcdb', 'OlumbeAKinkugbe', 'Nairametrics', 'followlasg', 'obyezeks', 'DrJoeAbah', 'UKinNigeria', 'SunshineFCAkure', 'MADENigerDelta', 'NsimaUEkere', 'LaureBeaufils', 'DAWNCommission', 'OndoStateHealth', 'SFHNigeria', 'floondo', 'D_Aruwajoye', 'Segunmimiko', 'realRochas', 'bonifacemwangi', 'LolaAkinmade', 'MNDA_Ng', 'dareynow', 'Realomosexy', 'daily_trust', 'ogundamisi', 'EiENigeria', 'SanRomeo', 'AATSarumi', 'bukolasaraki', 'PentagonMG', 'YeleSowore', 'NigerianLaw', 'NigBarAssoc', 'GovernorObaseki', 'FAdekoyaSAN', 'DeleMomodu', 'teamaketi', 'elrufai', 'OndoAPC', 'Adaba889fm', 'BukieAdetula', 'SenBoroffice', 'oandalegal', 'femigbaja', 'AkinwunmiAmbode', 'AlegehAugustine', 'APCNigeria', 'ProfOsinbajo', 'MBuhari', 'atiku', 'iamtokunbojames', 'adetai', 'AsiwajuTinubu', 'deluchi2001', 'abikedabiri', 'GbenroAdegbola', 'TRLPLAW', 'SundayDareSD', 'kunlf', 'looceye', 'Bettyaak', 'Elombah']

In [26]:
ayosogunro = ['ShineDarsh', 'thehassanian', 'dkieghe', 'chukaobi', 'Rene_noire', 'jacksonpbn', 'simonallison', 'slimsola', 'waweru', 'alea_iacta_', 'OlumideAkpata', 'ifedioku', 'theOluwatosin', 'shemmaye', 'Echecrates', 'haajara', 'kevinedavis', 'DocDre', 'hanit001', 'aprilwensel', 'NyawiraNjoroge', 'therealdaddymo1', 'NASA', 'MbalulaFikile', 'Ziziian', 'rulajebreal', 'KolaOyeneyin', 'CarolineKende', 'RealSolaKuti', 'thefaridaadamu', 'ChuksJaphet', 'ankaboy', '_IamWhispers', 'TrophyHusbandD', 'SCVNGoddess', 'Sylvia_Ikeji', 'UjuAnya', 'Chijioke_', 'ChuksUnfiltered', 'chris_nwosu_', 'chisom_hector', 'ennyola0015', 'Nimisire', 'vincentdesmond_', 'qwertybasss', 'Ayishat_Akanbi', 'davidoladeji', 'Donteewrites', 'Intergrity56', 'SusanIkegwu', 'eriifemusic', 'Iam_Yayi', 'light_blazer', 'WilsonBrownYomi', 'toluxylic', 'PelumiK', 'Main_guy_', 'ubongabasi', 'Femi_Areola', 'funshographix', 'espeezeal', 'Senator_Chegxie', 'aweleemili', 'temilolu_wah', 'justbeing_ayo', 'tritimaa_', 'swemyada', 'bowlofcrisps', 'JUSTADACHI', 'article19law', 'fqdayib', 'EiENigeria', 'chosensomto', 'eldivyn', 'redlightvoices', 'Stainless247', 'faithmight', 'jamiltoyo', 'sabimson', 'FLIPTYCE', 'nazaokoli', 'nicd_sa', 'onuoraogunoac', 'duchesskk', 'sinmbo', 'Spidey77949623', 'Baldilocks__', 'OleItumbi', 'UtaMakhema', 'k_odada', 'animolenikun', 'the_odditty', 'njokingumi', 'michellechivung', 'NCDCgov', 'hrw', 't0nit0ne', 'MoghaluKingsley', 'adamugarba', 'sugeezy', 'haddijatoujonga', 'LadyEumukoro', 'investbamboo', 'MucahidDurmaz', 'MohamedFall', 'MsLadyPhyll', 'tazmus982', 'SolomonADersso', 'Riddwane', 'ShalomDickson', 'PanAfricanUnity', 'K_AdurojaTVC', 'Marketplaces247', 'oryembley', 'fisayosoyombo', 'oluwaloninyo', 'BigDaddyonair', 'ChukwuderaEdozi', 'rrudegal', 'wfm917', 'kikimordi', 'realREANigeria', 'dounardbondo', 't3niola', 'gechife', 'AjiSaineyKahESQ', 'DokunOjomo', 'profdanhicks', 'MelusiSiboniso', 'mataba02', 'DrCOmole', 'BTDTHub', 'PalmerReport', 'Tshada_M', 'jack', '_MrsMalaprop', 'LaureBeaufils', 'BankyW', 'joeloseiga', 'Adebo_ogund', 'gossyomega', 'JoBaidoo', 'thelifeofdami', 'DamzAdeleye', 'NekkaSmith', 'tate_musinahama', 'ayoolids', 'kachi208', 'conumah', 'gentleojay', 'OhTimehinSays', 'CanHCNigeria', 'bankulli', 'kudakwa93753794', 'ruthmaclean', 'tunjiomotola', 'JohnCampbellcfr', 'bisiafayemi', 'debbidooda', 'eyderp', 'LaurieAClaase', 'jessemsperling', 'Elle_Drews', 'HassanIdayat', 'Jollz', 'EmekaAmakeze', 'DavidHundeyin', 'Stylebymixie', 'JanetUosu', 'tino_wino', 'DidiNtheBanx', 'ulxma', 'sharon_ikeazor', 'donnasthill', 'louis_ntaj', 'brandsofcourage', 'AE_Yamin', 'aoteh', 'bakalir1', 'Muciru', 'Ay_Pea', 'soledadobrien', '_nthatii_', 'sheetal6189', 'iamseunbakare', 'MagnusKillander', 'ProfLizGriffin', 'iyad_elbaghdadi', '_honeyricci_', 'Lazeewriter', 'KayaFMTalk', 'CHLemchi', 'johnedwinmason', 'maleeki_', 'HarryItie', 'aonanuga1956', 'NeelaGhoshal', 'olatoxx', 'tolujonson', 'afureo', 'Ige007', 'Oludidi', 'BiggiePinGing', 'ozzyetomi', 'akorive001', 'aniewang', 'SashaPofficial', 'dessyslays1', 'xkape', 'chumathomp', 'AEinitiatives', 'KateOsamor', '_torinmo', 'seunonigbinde', 'OmogeDami', 'aushamaki', 'andyRoidO', 'marie_mlola', 'BoluwajiDavids', 'AdakuUfere', 'AbeOyeniyi', 'MatthewWaites', 'JLPisJLPisJLP', 'Thedammylawal', 'dejiajare', 'FayWangari', 'inpoco', 'LeahFessler', 'remyngol', 'IAchpr', 'fannygomezlugo', 'achiengakena', 'oghalebu', 'Citizens4Africa', 'enobong', 'afalli', 'DiscretionUiB', 'ybnl_princess', 'GarriCake', 'Kingwole', 'lordRij', 'Lindzilia', 'nellylaoni', 'nals14', 'llsisi', 'Mzlianne', 'Umars_Daughter', 'OlufemiAwoyemi', 'rolakebamgbose', 'madayo', 'clivesimpkins', 'SheikhaSarwi', 'debbieo1_', 'ToyinSaraki', 'geokaycee', 'TundeIrukera', 'vilebillionaire', 'briticoyemo', 'mariamkmarr', 'MsKhaddie', 'ChifeDr', 'AusHCAbuja', 'henryshield', 'waithera_jk', 'olaokun_s', 'HenshawKate', 'philhazlewood', 'seye', 'UNCLE_AJALA', 'HamzyCODE', 'TheYomiKazeem', 'bellanaija', 'rakewalaw', 'musubakoto', 'ifenihinlola', 'tosinolaseinde', 'tomiwa_ilori', 'NFF2008', 'JeleCynthia', 'gbemy01', 'AcjaObs', 'GuardianNigeria', 'Warega_N', 'ciaramoconnell', 'ACHPRreport', 'LawTransform', 'ZumaXY', 'AyoBankole', 'HafsatKIND', 'subomiplumptre', 'innosonvehicles', 'NonieMG', 'S_khumalo', 'GambiaNYC', 'YeleSowore', 'OmololaOmotoso1', 'SJFindlay', 'aadetugbo', '_lamide_', 'siobhan_ogrady']

In [27]:
funkeakindele = ['Lucybab57879420', 'Bethesda_school', 'Walexdhino', 'Iam_IkeOnyema', 'theNETng', 'RTLifeQuotes', 'ThisInspiresUs', 'IsholaOlufemi14', 'ThatPastorzKid', 'iam_westhiez', 'abimbolabmw', 'Beccabae3', 'oorebambam', 'Iam_Olusanya', 'queen_vbeno', 'Dlolahh', 'Phunmilahyoh', 'odavidinho30', 'iam_hyrish', 'JAYNE_CLAIRE__', 'tarajiphenson', 'everest', 'Happine92878457', 'muftimenk', 'iam_vanvicker', 'crazeclown', 'Zlatan_Ibile', 'SamAdegoke', 'officialnairam1', 'AiyetoroTown', 'stylebender', 'MoAbudu', 'anythingoye', 'FakhuusHashim', 'CleannaijaNG', 'adekunleGOLD', 'ActionAidNG', 'iamkissdaniel', 'yemoka', 'Attanda_', 'fchibuzo23', 'FolarinXO', 'Iyanya', '2niteFlavour', 'official2baba', 'Dharmiee01', 'thesimesty', 'adeyemioyinda12', 'yemialadee', 'Teanuhcouture', 'Dammy_eneli', 'Rameseck', 'KunleJONISE', 'gbolahanodebowa', 'IWV', 'obaxlove87', 'AmbroseOkwoli', 'omojolaDamilol7', 'kunbzberry', 'PrettyNenny89', 'mimir0wlz', 'dayoslides', 'Latestoby', 'msutaay', 'tiffany_ehi', 'Westernlottong', 'Yemisi__O', 'iam_teem', 'oluwakemi200', 'DJSPINALL', 'Wazobia_FM', 'zack_aminu', 'obiasika', 'OBOGlobal', 'PaolaAudrey', 'AgencePannelle', 'real__stella', 'kollybhadmus', 'bolanleolukanni', 'MTVBaseWest', 'CoolFMNigeria', 'imoheboh', 'jamesbacon769', 'LindaEjiofor', 'Industreetdrama', 'TheLotaChukwu', 'poga09', 'Oluwatoyosiest', 'SAMKLEF', 'LifeOfDorbulyn', 'preciouseghieye', 'busifunmi', 'iamadunniade', 'Off_Gemstones', 'iamMoEazy', 'Bubbles_O', 'THEBEAT999FM', 'HproMovies', 'sonorouskanye_', 'MartinsFeelz', 'burnaboy', 'Obistical1', 'soprecords_', 'fluxyangel', 'MuyiwaAkinlehin', 'Omojuwa', 'Haleemah____', 'temithorpe_242', 'wisdom2inspire_', 'JaiKaliMa_', 'Joanna__', 'InspiredUs', 'MuyiMyghty', 'aphroDIJA', 'obyezeks', 'circesfx', 'ChidimmaChidim8', 'olayiwo24635279', 'topeoshin', '_Arikeade_', 'Mz_tunmie', 'Miz_Fey', 'Iamhemmanuel', 'amenestate', 'omonagbator', 'TheCooksCook', 'A_shadeof_blaq', 'akahnnani', 'gbengaaruleba', 'PastorID', 'RCCGCityofDavid', 'Aadetola_', 'TWEETORACLE', 'ericyeboahgh', 'DJAMEDIA', 'lausteves', 'NTAMoviesToday', 'Dorlipi', 'SceneoneTV', 'IamMcMallam01', 'diva_nehneh', 'tashasmith4real', 'KingFoye_', 'OfficialAyuba', 'Evaebonyy', 'loy_loyjack', 'LebaraPlay', 'Yvethi', 'RealChinenye', 'NoelleFranc3sca', 'modebayooni', 'whema', 'OgbeniDipo', 'Priyanka_africa', 'DaTolu_Fash', 'bolaallison', '3ltino', 'TanyaVome', 'deejayskotty', 'oluwatobiajatta', 'Akinsanyadanie2', 'iam_Tobiismail', 'TobiMoshood', 'hormorte', 'MarioMonforte', 'TosynBucknor', 'koolblaque', 'SirLogix', 'lanrry001', 'BettyIrabor', 'GodwinAruwayo', 'ushbebecomedian', 'sabinadgreat', 'dauda_suzana', 'wanny4show', 'Chief_di234', 'philips_Nonny', 'glorykech', 'alvanchinaka', 'RealisticGirlie', 'employtheworld', 'irokotv', 'Sumzyhotbabe', 'AbisHappyPlace', 'TaymiB', 'MotivatedLiving', 'TeamKateHenshaw', 'shane_seun', 'UpliftingQuotes', 'motivational', 'Quotes_Life', 'GreatestQuotes', 'StumblerBest', 'yoruba_proverbs', 'sleekiee', 'EarlofLagos', 'EddieLeem', 'OfficialMissKDK', 'princessijay1', 'COOLP14', 'nenegwill', 'AnyanwuEminent', 'datcoolgaffy', 'gbomolewa', 'benlandis', 'africamagictv', 'Ximena_HUD', 'funket4u', 'OyindamolaDada', 'AdetoyeseAliyu', 'osinachicp', 'Bigpsamy', 'babynass1', 'TommyPusso101', 'ijeomajokafor', 'OfficerRod8037', 'KameronBennett', 'jos_pretty', 'GbemiOO', 'KubyUyanga', 'AYCOMEDIAN', 'MattPoss', 'abiolaasekun', 'fundebra', 'funnyogbolor', 'OloriSupergal', 'Lamboginny', 'fakuadefhemmy', 'khleorozy', 'Sir3lvis', 'shesooshants', 'SamadeyemiA', 'AdedejAdedeji', 'Gentleexponent', 'comphie01', '6BillionPeople', 'Iam_batungas', 'Kingblazey', '0lumuyiwac0ker', 'iyaagba', 'ObinnaOfficial', 'OlaosunD', 'realpower32', 'lanrewaju_dayo', 'realokuchukwu', 'Saltofdworld', 'jummy_lee', 'PinkPearlNG', 'Tom_dopezzy', 'atone_fola', '35752baec6e940a', 'WirelessIgbokwe', 'mistacooliyoo', 'dakolalads', 'EstolPropsMedia', 'Funmi4PH', 'taiwo_somefun', 'Jessejagz', 'haybydeymie', 'gbadegshine10', 'Olamide', 'IamDrSID', 'bizzleosikoya', 'abokithebahdguy', 'IamMisfitz', 'falzthebahdguy', 'GbemiLukman', 'dapheonixking', 'bukola_abel', '_Torha', '937RhythmFm', 'GodyBeny', 'DebolaLagos', 'Oluwafisher', 'FLIPTYCE', 'yakubumohammed_', 'nikky_susan', 'namazzing', 'jenifasdiary', 'InkMaitres', 'Akinbode2Tobi', 'Sheilapearel', 'Redimz_', 'MTVBaseAfrica', 'ANITAOLUGHU', 'jazzybayo', '_Bubblebutt1', 'PrincessIjay', 'cheeeomaah', 'Yemisi_K', 'shawttynatt', 'ug_ehumadu', 'moyosibe', 'salaudeensefiu1', 'SoguAyodeji', 'waliuabayomi', 'harhbeesolar', 'Stezzzzzzzz', 'GodPikin_CFR', 'itz_cassbae']

In [28]:
topealabi01 = ['MahoganyHnS', 'rccghq', 'TeniEntertainer', 'realFemiOtedola', 'cobhamsasuquo', 'realDonaldTrump', 'JoelOsteen', 'channelstv', 'SympLySimi', 'mikearemu1', 'IAdefarasin', 'ft_canaanland', 'onosariyo', 'DayoAmusa', 'HenshawKate', 'tkelani', 'funkeakindele', 'NCDCgov', 'AsiwajuTinubu', 'glowreeyah', 'AdaEhiMoses', 'NikkiLaoye', 'eben_rocks', 'sammieokposo', 'timgodfreyworld', 'houseontherock', 'pauladefarasin', 'donmoen', 'TheExperienceLG', 'iamchiomajesus', 'TopeAlabi15', 'NikeAdeyemi', 'sam_adeyemi', 'TravisGreeneTV', 'JekalynCarr', 'nathanielblow', 'tybellotweets', 'MatAshimolowo', 'KemiOlunloyo', 'FemiHamzat', 'drobafemihamzat', 'drmikeokonkwo', 'UBAGroup', 'ZenithBank', 'sinach', 'ayomikuu', 'Buchibwai', 'Midnightcrewng', 'patuwajeking', 'mqmonique', '__debiroo__', 'mikeabdulng', 'FaithOyedepo', 'pastortaiwo', 'DavidOyedepoMin', 'ProfOsinbajo', 'BarackObama', 'yawazobiafm', 'jidesanwoolu', 'AkinwunmiAmbode', 'pastorpoju', 'CNN', 'cnnbrk', 'NigeriainfoFM', 'MahoganyHnS', 'rccghq', 'TeniEntertainer', 'realFemiOtedola', 'cobhamsasuquo', 'realDonaldTrump', 'JoelOsteen', 'channelstv', 'SympLySimi', 'mikearemu1', 'IAdefarasin', 'ft_canaanland', 'onosariyo', 'DayoAmusa', 'HenshawKate', 'tkelani', 'funkeakindele', 'NCDCgov', 'AsiwajuTinubu', 'glowreeyah', 'AdaEhiMoses', 'NikkiLaoye', 'eben_rocks', 'sammieokposo', 'timgodfreyworld', 'houseontherock', 'pauladefarasin', 'donmoen', 'TheExperienceLG', 'iamchiomajesus', 'TopeAlabi15', 'NikeAdeyemi', 'sam_adeyemi', 'TravisGreeneTV', 'JekalynCarr', 'nathanielblow', 'tybellotweets', 'MatAshimolowo', 'KemiOlunloyo', 'FemiHamzat', 'drobafemihamzat', 'drmikeokonkwo', 'UBAGroup', 'ZenithBank', 'sinach', 'ayomikuu', 'Buchibwai', 'Midnightcrewng', 'patuwajeking', 'mqmonique', '__debiroo__', 'mikeabdulng', 'FaithOyedepo', 'pastortaiwo', 'DavidOyedepoMin', 'ProfOsinbajo', 'BarackObama', 'yawazobiafm', 'jidesanwoolu', 'AkinwunmiAmbode', 'pastorpoju', 'CNN', 'cnnbrk', 'NigeriainfoFM']

In [29]:
channelstv = ['hadisirika', 'thesunnigeria', 'BBCNews', 'AJEnglish', 'seyiamakinde', 'officialKolaO', 'MKKyari', 'ProfAkinAbayomi', 'RotimiAkeredolu', 'dabiodunMFR', 'thecableng', 'LSMOH', 'NigeriaMFA', 'gboyegaakosile', 'Fmohnigeria', 'WHONigeria', 'NCDCgov', 'NGRPresident', 'CTVKaleidoscope', 'CatrionaLaing1', 'DailyMailUK', 'mailandguardian', 'dwnews', 'XHNews', 'WSJ', 'globaltimesnews', 'TIME', 'TechCrunch', 'ChibuikeAmaechi', 'AFP', 'WIRED', 'NewYorker', 'nytimes', 'FAAN_Official', 'FMoCDENigeria', 'sgyemikale', 'akandeoj', 'ICPC_PE', 'FMHDSD', 'neweconforum', 'HouseNGR', 'femigbaja', 'SpeakerGbaja', 'GavelAbuja', 'cenbank', 'nidcom_gov', 'followlastma', 'SPNigeria', 'ChannelsAcademy', 'GEJonathan', 'UNICEF', 'UN', 'SituationRoomNg', 'TrackaNG', 'BBCWorld', 'aledeh', 'nigimmigration', 'KayodeOkikiolu', 'BBCAfrica', 'ChannelsTVPress', 'PoliceNG_CRU', 'atiku', 'jidesanwoolu', 'jimiagbaje', 'FRSCNigeria', 'USinNigeria', 'inecnigeria', 'ctvnewstracker', 'followlasg', 'WorldBankAfrica', 'GuardianNigeria', 'OfficialAPCNg', 'thenff', 'TheChatCTV', 'mitchokonta', 'LadiAAle', 'CTVRoadMap', 'NgComCommission', 'SERAPNigeria', 'JAMBHQ', 'NigeriaGov', 'nassnigeria', 'NNPCgroup', 'Ronkejewels', 'OkeyBakassy', 'TheOtherNewsCTV', 'CTVAFL', 'kduranta', 'OyebisiAdebayo', 'JosephEsenwa', 'AmbroseOkohCTV', 'CTV_TayoS', 'uchealaukwu', 'Holarbowaley', 'BudgITng', 'YNaija', 'GovKaduna', 'ShehuSani', 'kfayemi', 'GarShehu', 'GovAyoFayose', 'nemanigeria', 'NOA_Nigeria', 'ijeomaonyeator', 'realDonaldTrump', 'ladeyemi', 'ievbuomwan', 'newkaypun', 'officialEFCC', 'CTVHardcopy', 'CNNAfrica', 'BJRewane', 'EiENigeria', 'ronke_raji', 'millicentetim', 'OfficialPDPNig', 'mariabyrdctv', 'YemisiIpayeCTV', 'CTV_ceceO', 'coemuze', 'LeadershipNGA', 'daily_trust', 'DailyPostNGR', 'obyezeks', 'TheNationNews', 'headbayo', 'visionaria_01', 'Betty_Dibiah1', 'CTV_MaryA', 'CTVEnvironment', 'dw_environment', 'NGSuperEagles', 'jumiebabs', 'AkinwunmiAmbode', 'Mckbram', 'OfficialMiccolo', 'NGRSenate', 'GimbaUmarCTV', 'KayAlayande', 'taiwoadeshina_', 'ayoolakassim', 'MelindaAkinlami', 'eaopara', 'sunriseCTV', 'channelsbooks', 'PhilipsOlu', 'faceofftoday', 'PoliceNG', 'bukolasaraki', 'ProfOsinbajo', 'AnthoniaOrji', 'CTV_Austin', 'ctv_ilemsc', 'CTV_AbidemiD', 'lucillious', 'Anne_Nwaghodoh', 'MaupeO', 'UKinNigeria', 'SolaMomoh', 'yomi_otaigbe', 'iWitnessCTV', 'MBuhari', 'itempili', 'DefenceInfoNG', 'HQNigerianArmy', 'boasonbusiness', 'joomomoh', 'chrisilems', 'charliediceman', 'CTVpolitics', 'channelstvbiz', 'channels_sports', 'NERCNG', 'ToyinIbitoye', 'Amarachi_Ubani', 'metrofilectv', 'kenochonogor', 'seunokin', 'ChannelsBeam', 'victor_mbidi', 'Channels24_UK', 'datelineabuja', 'llsisi', 'KayodeAkintemi', 'PremiumTimesng', 'inivictoria', 'chamberlainusoh', 'ogechukwuosih', 'nneotaegbe', 'iamshola', 'pmnewsnigeria', 'SaharaReporters', 'MobilePunch', 'vanguardngrnews', 'sunrisedailynow', 'CNN', 'Reuters', 'BBCBreaking', 'cnnbrk', 'hadisirika', 'thesunnigeria', 'BBCNews', 'AJEnglish', 'seyiamakinde', 'officialKolaO', 'MKKyari', 'ProfAkinAbayomi', 'RotimiAkeredolu', 'dabiodunMFR', 'thecableng', 'LSMOH', 'NigeriaMFA', 'gboyegaakosile', 'Fmohnigeria', 'WHONigeria', 'NCDCgov', 'NGRPresident', 'CTVKaleidoscope', 'CatrionaLaing1', 'DailyMailUK', 'mailandguardian', 'dwnews', 'XHNews', 'WSJ', 'globaltimesnews', 'TIME', 'TechCrunch', 'ChibuikeAmaechi', 'AFP', 'WIRED', 'NewYorker', 'nytimes', 'FAAN_Official', 'FMoCDENigeria', 'sgyemikale', 'akandeoj', 'ICPC_PE', 'FMHDSD', 'neweconforum', 'HouseNGR', 'femigbaja', 'SpeakerGbaja', 'GavelAbuja', 'cenbank', 'nidcom_gov', 'followlastma', 'SPNigeria', 'ChannelsAcademy', 'GEJonathan', 'UNICEF', 'UN', 'SituationRoomNg', 'TrackaNG', 'BBCWorld', 'aledeh', 'nigimmigration', 'KayodeOkikiolu', 'BBCAfrica', 'ChannelsTVPress', 'PoliceNG_CRU', 'atiku', 'jidesanwoolu', 'jimiagbaje', 'FRSCNigeria', 'USinNigeria', 'inecnigeria', 'ctvnewstracker', 'followlasg', 'WorldBankAfrica', 'GuardianNigeria', 'OfficialAPCNg', 'thenff', 'TheChatCTV', 'mitchokonta', 'LadiAAle', 'CTVRoadMap', 'NgComCommission', 'SERAPNigeria', 'JAMBHQ', 'NigeriaGov', 'nassnigeria', 'NNPCgroup', 'Ronkejewels', 'OkeyBakassy', 'TheOtherNewsCTV', 'CTVAFL', 'kduranta', 'OyebisiAdebayo', 'JosephEsenwa', 'AmbroseOkohCTV', 'CTV_TayoS', 'uchealaukwu', 'Holarbowaley', 'BudgITng', 'YNaija', 'GovKaduna', 'ShehuSani', 'kfayemi', 'GarShehu']

In [30]:
combined = channelstv + topealabi01 + funkeakindele + RotimiAkeredolu + seyiamakinde + ayosogunro + extracted_newslink + top_govOfficials

In [31]:
len(combined)

1705

In [32]:
mylist = list(dict.fromkeys(combined))

In [33]:
len(mylist)

1155

In [ ]:
tweets = []
    
for x in range(0, len(mylist)):

    # The Twitter user who we want to get tweets from
    name = mylist[x]

    # Calling the user_timeline function with our parameters
    results = api.user_timeline(id=name, count=500)

    # foreach through all tweets pulled
    for tweet in results:
    # printing the text stored inside the tweet object
        tweets.append(tweet.text)


In [35]:
len(tweets)

72135

In [36]:
from pandas import DataFrame
df = DataFrame (tweets,columns=['Tweets of Gotten Handles'])
df

,Tweets of Gotten Handles
0,International Flight resumption date is not Oc...
1,RT @IATA: #DYK: HEPA filters on planes play an...
2,My colleagues &amp; I have reviewed passenger ...
3,🕺🏽🕺🏽🕺🏽 https://t.co/jDsrlHnjmA
4,Our new normal!🙏🏽🤲🏽🇳🇬 https://t.co/gkqItUD8Vg
...,...
72130,JUST IN: Tiger Woods just clinched his fifth M...
72131,@StellaDamasus stuns at the London premiere of...
72132,Photos: @StellaDamasus stuns at the London pre...
72133,Stella Damasus stuns at the London premiere of...


In [56]:
import csv

with open(r'C:\Users\Tijesunimi2-PC\Documents\10academy_week2_challenge\tweets.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(tweets)